In [1]:
## Parameters
fold=1
dataset="monks-1"
D = 2# depth


In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 18 22:11:32 2023

@author: khaniyev
"""

#%%

import gurobipy as gp

import gurobipy
import pandas as pd
import numpy as np

relpath="./Datasets/"+dataset+"/fold="+str(fold)+"_train.csv"
data = pd.read_csv(relpath)#pd.read_csv("./Datasets/fold=1_train.csv")

X = data.iloc[:,1:]
y = data.iloc[:,:1]

PP = [i for i in range(data.shape[1]-1)]
P = len(PP)

KK = list(data.y.unique())
K = len(KK)

DD = list(range(D))
L = 2**D
LL = list(range(L))

NN = list(range(X.shape[0]))
N = len(NN)

y_k = np.zeros((X.shape[0], K))
for k in range(K):
    y_k[np.where(y==KK[k])[0],k] = 1
x = np.array(X)

relpath="./Datasets/"+dataset+"/fold="+str(fold)+"_test.csv"
data_test = pd.read_csv(relpath)#pd.read_csv("./Datasets/fold=1_train.csv")

X_test = data_test.iloc[:,1:]
y_test = data_test.iloc[:,:1]

y_k_test = np.zeros((X_test.shape[0], K))
for k in range(K):
    y_k_test[np.where(y_test==KK[k])[0],k] = 1

y_k_test=y_k_test.astype(np.int32)
x_test = np.array(X_test,dtype=np.int32)





relpath="./Datasets/"+dataset+"/fold="+str(fold)



In [3]:
import numpy as np
from typing import Tuple

def get_split_values(leaf_type: int, depth: int) -> np.ndarray:
    """
    Compute the sequence of split values (inverted binary digits) for a given leaf type.

    Parameters
    ----------
    leaf_type : int
        Integer encoding of the leaf path (as a binary number).
    depth : int
        Number of levels (bits) in the path.

    Returns
    -------
    split_values : np.ndarray of shape (depth,)
        Array of floats in {0.0, 1.0}, where each entry is
        1.0 minus the corresponding bit of leaf_type (MSB first).
    """
    # Extract bits from most‐significant to least‐significant
    bits = [
        (leaf_type >> (depth - l - 1)) & 1
        for l in range(depth)
    ]
    # Invert bits and cast to float
    split_values = np.array([1.0 - b for b in bits], dtype=float)
    return split_values


import numpy as np
from typing import Sequence, Tuple

def get_misclassification(
    x: np.ndarray,
    y_k: np.ndarray,
    leaf_pattern: Sequence[int],
    leaf_type: int
) -> Tuple[float, int, np.ndarray]:
    """
    Compute misclassification count over the subset of rows matching a leaf pattern.
    
    Parameters
    ----------
    x : np.ndarray of shape (n_samples, n_features)
        Feature matrix (integer-coded).
    y_k : np.ndarray of shape (n_samples, n_classes)
        One‐hot or count matrix of true labels/classes.
    leaf_pattern : sequence of int
        Indices of features to test at each depth.
    leaf_type : int
        Identifier passed to get_split_values.
    
    Returns
    -------
    misclassification : float
        num_samples - max_class_count within the filtered subset.
    num_samples : int
        Number of rows matching the leaf test.
    mask : np.ndarray of bool, shape (n_samples,)
        Boolean mask selecting rows that satisfy all splits.
    """
    depth = len(leaf_pattern)
    # get_split_values should return a length‐depth array of split thresholds
    split_values = get_split_values(leaf_type, depth)

    # Start with all True, then AND‐mask down to those matching every split
    mask = np.ones(x.shape[0], dtype=bool)
    for l, feat_idx in enumerate(leaf_pattern):
        mask &= (x[:, feat_idx] == split_values[l])

    # Filtered labels
    y_filt = y_k[mask]
    num_samples = y_filt.shape[0]

    if num_samples == 0:
        return 0.0, 0, mask

    # Sum up counts per class and pick the largest
    class_sums = y_filt.sum(axis=0)
    max_class_sum = class_sums.max()

    misclassification = num_samples - max_class_sum
    return misclassification, num_samples, mask


In [4]:
def get_tree_mis(inc_patterns,d,x,y_k):
    miss=0
    for leaf in range(2**d):
        miss+=get_misclassification(x, y_k, inc_patterns[leaf], leaf)[0]
    return miss/len(x)
    
    

In [5]:



if D==2:
    oct_dw_2 = gp.Model()
    oct_dw_2.setParam(gp.GRB.Param.Seed,42)
    oct_dw_2.setParam("OutputFlag",1)
    oct_dw_2.setParam('TimeLimit', 3600)
    z = oct_dw_2.addVars(P, D, L, vtype=gp.GRB.BINARY,name='z')
    t = oct_dw_2.addVars(N, L, vtype=gp.GRB.CONTINUOUS, ub=1)
    c = oct_dw_2.addVars(K, L, vtype=gp.GRB.BINARY)

    
    misclassification_error = gp.LinExpr((1/N)*gp.quicksum(t[i,l] for i in range(N) for l in range(L)))
    oct_dw_2.setObjective(misclassification_error, gp.GRB.MINIMIZE)
    
    oct_dw_2.addConstrs(gp.quicksum(z[j,d,l] for j in range(P)) == 1 for l in range(L) for d in range(D)) 
    
    l=0
    oct_dw_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*z[j,0,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,1,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=1
    oct_dw_2.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*z[j,0,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,1,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=2
    oct_dw_2.addConstrs(t[i,l] >= gp.quicksum((1-x[i,j])*z[j,0,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,1,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=3
    oct_dw_2.addConstrs(t[i,l] >= gp.quicksum((1-x[i,j])*z[j,0,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,1,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    oct_dw_2.addConstrs(gp.quicksum(c[k,l] for k in range(K)) == 1 for l in range(L)) 
    
    oct_dw_2.addConstrs(z[j,0,0] == z[j,0,1] for j in range(P))
    oct_dw_2.addConstrs(z[j,1,0] == z[j,1,1] for j in range(P))
    oct_dw_2.addConstrs(z[j,0,2] == z[j,0,3] for j in range(P))
    oct_dw_2.addConstrs(z[j,1,2] == z[j,1,3] for j in range(P))
    oct_dw_2.addConstrs(z[j,0,0] == z[j,0,2] for j in range(P))
    
    oct_dw_2.update()
    # oct_dw_lp = oct_dw.relax()
    opt_sol = oct_dw_2.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-26
Set parameter Seed to value 42
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 2087 rows, 2128 columns and 34278 nonzeros
Model fingerprint: 0x72ca0a4e
Variable types: 2000 continuous, 128 integer (128 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-03, 2e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 0.4120000
Presolve removed 476 rows and 471 columns
Presolve time: 0.02s
Presolved: 1611 rows, 1657 columns, 27381 nonzeros
Variable types: 1608 continuous, 49 integer (49 binary)

Root relaxation: objective 0.000000e+00, 100 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node 

In [6]:
#%% Depth = 3
if D==3:
    oct_dw_d3 = gp.Model()
    oct_dw_d3.setParam("OutputFlag",1)
    oct_dw_d3.setParam('TimeLimit', 3600)
    oct_dw_d3.setParam(gp.GRB.Param.Seed,42)
    
    z = oct_dw_d3.addVars(P, D, L, vtype=gp.GRB.BINARY,name='z')
    t = oct_dw_d3.addVars(N, L, vtype=gp.GRB.CONTINUOUS, ub=1)
    c = oct_dw_d3.addVars(K, L, vtype=gp.GRB.BINARY)
    
    misclassification_error = gp.LinExpr((1/N)*gp.quicksum(t[i,l] for i in range(N) for l in range(L)))
    oct_dw_d3.setObjective(misclassification_error, gp.GRB.MINIMIZE)
    
    oct_dw_d3.addConstrs(gp.quicksum(z[j,d,l] for j in range(P)) == 1 for l in range(L) for d in range(D)) 
    
    l=0
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*z[j,0,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,1,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=1
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*z[j,0,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,1,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=2
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*z[j,0,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,1,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=3
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum(x[i,j]*z[j,0,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,1,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=4
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum((1-x[i,j])*z[j,0,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,1,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=5
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum((1-x[i,j])*z[j,0,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,1,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=6
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum((1-x[i,j])*z[j,0,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,1,l] for j in range(P)) + gp.quicksum(x[i,j]*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    l=7
    oct_dw_d3.addConstrs(t[i,l] >= gp.quicksum((1-x[i,j])*z[j,0,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,1,l] for j in range(P)) + gp.quicksum((1-x[i,j])*z[j,2,l] for j in range(P)) - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K)) - (D-1) for i in range(N))  
    
    
    oct_dw_d3.addConstrs(gp.quicksum(c[k,l] for k in range(K)) == 1 for l in range(L)) 
    
    ### Siblings ###
    oct_dw_d3.addConstrs(z[j,0,0] == z[j,0,1] for j in range(P))
    oct_dw_d3.addConstrs(z[j,1,0] == z[j,1,1] for j in range(P))
    oct_dw_d3.addConstrs(z[j,2,0] == z[j,2,1] for j in range(P))
    
    oct_dw_d3.addConstrs(z[j,0,2] == z[j,0,3] for j in range(P))
    oct_dw_d3.addConstrs(z[j,1,2] == z[j,1,3] for j in range(P))
    oct_dw_d3.addConstrs(z[j,2,2] == z[j,2,3] for j in range(P))
    
    oct_dw_d3.addConstrs(z[j,0,4] == z[j,0,5] for j in range(P))
    oct_dw_d3.addConstrs(z[j,1,4] == z[j,1,5] for j in range(P))
    oct_dw_d3.addConstrs(z[j,2,4] == z[j,2,5] for j in range(P))
    
    oct_dw_d3.addConstrs(z[j,0,6] == z[j,0,7] for j in range(P))
    oct_dw_d3.addConstrs(z[j,1,6] == z[j,1,7] for j in range(P))
    oct_dw_d3.addConstrs(z[j,2,6] == z[j,2,7] for j in range(P))
    
    ### First Cousins ###
    
    oct_dw_d3.addConstrs(z[j,0,0] == z[j,0,2] for j in range(P))
    oct_dw_d3.addConstrs(z[j,1,0] == z[j,1,2] for j in range(P))
    
    oct_dw_d3.addConstrs(z[j,0,4] == z[j,0,6] for j in range(P))
    oct_dw_d3.addConstrs(z[j,1,4] == z[j,1,6] for j in range(P))
    
    ### Second Cousins ###
    
    oct_dw_d3.addConstrs(z[j,0,0] == z[j,0,4] for j in range(P))
    
    oct_dw_d3.update()
    # oct_dw_lp = oct_dw.relax()
    opt_sol = oct_dw_d3.optimize()
    

In [7]:
#%% Depth = 4

if D==4:
    oct_dw_d4 = gp.Model()
    oct_dw_d4.setParam("OutputFlag",1)
    oct_dw_d4.setParam('TimeLimit', 3600)
    oct_dw_d4.setParam(gp.GRB.Param.Seed,42)

    L = 2**D   # 16 leaves

    z = oct_dw_d4.addVars(P, D, L, vtype=gp.GRB.BINARY, name='z')
    t = oct_dw_d4.addVars(N, L, vtype=gp.GRB.CONTINUOUS, ub=1, name='t')
    c = oct_dw_d4.addVars(K, L, vtype=gp.GRB.BINARY, name='c')

    # objective
    misclassification = (1/N) * gp.quicksum(t[i,l] for i in range(N) for l in range(L))
    oct_dw_d4.setObjective(misclassification, gp.GRB.MINIMIZE)

    # 1) exactly one predictor chosen at each (d,l)
    oct_dw_d4.addConstrs(
        gp.quicksum(z[j,d,l] for j in range(P)) == 1 
        for d in range(D) for l in range(L)
    )

    # 2) misclassification (t) constraints
    for l in range(L):
        bits = [ (l >> (D-1-d)) & 1 for d in range(D) ]
        for i in range(N):
            expr = gp.quicksum(
                ( x[i,j]   if bits[d]==0 else (1-x[i,j]) ) * z[j,d,l]
                for d in range(D) for j in range(P)
            )
            oct_dw_d4.addConstr(
                t[i,l] >= expr
                         - gp.quicksum(y_k[i,k]*c[k,l] for k in range(K))
                         - (D-1)
            )

    # 3) one leaf‐classifier per leaf
    oct_dw_d4.addConstrs(
        gp.quicksum(c[k,l] for k in range(K)) == 1
        for l in range(L)
    )

    # 4) enforce z‐consistency under each internal node
    for d in range(D):
        block_size = 2**(D-d)
        num_nodes  = 2**d
        for m in range(num_nodes):
            leaves = range(m*block_size, (m+1)*block_size)
            l0 = leaves[0]
            for l in leaves[1:]:
                oct_dw_d4.addConstrs(
                    z[j,d,l] == z[j,d,l0]
                    for j in range(P)
                )

    oct_dw_d4.update()
    opt_sol = oct_dw_d4.optimize()


In [8]:
model= oct_dw_2 if D==2 else oct_dw_d3 if D==3 else oct_dw_d4

In [9]:
nonzero=np.array(model.getVars())[np.where(np.array(model.X[:P*D*L])==1)]
var_names = [v.VarName for v in nonzero]

patterns=[['' for d_ in range(D)] for i in range(L)]
for var_name in var_names:
    # 1) grab the part inside the brackets
    inside = var_name[var_name.find('[')+1 : -1]    # '9,1,2'
    # 2) split on commas and convert to ints
    feature, depth, leaf = map(int, inside.split(','))
    # 3) assign
    patterns[leaf][depth] = feature
# TEST DATASET
miss_test=get_tree_mis(patterns,D,x_test,y_k_test)

# combine res
res=np.array([1-model.objval,1-miss_test,model.getAttr("Runtime"),model.MIPGap])

labels=np.array(['Training Accuracy','Test Accuracy','Time','Opt Gap'])
resWlabel=np.vstack([labels,res]).T

np.savetxt(relpath+'_resCompactOrdered_'+str(D)+'.txt',resWlabel,delimiter=',',fmt='%s')


In [10]:
import sys

orig_stdout = sys.stdout
f = open(relpath+'_DecisionRules_OrderedCompact_'+str(D)+'.txt', 'w')
sys.stdout = f

print(patterns)

sys.stdout = orig_stdout
f.close()